# Latin Hyper Cube Sampling
* Performs Sampling for Nas101

In [1]:
from os import path
import sys
print(path.abspath(path.abspath('../../')))
sys.path.append(path.abspath(path.abspath('../../')))

/Users/safayilmaz/Desktop/DI LAB/NASLib


* We need naslib101 api to use the provided code. 

In [2]:
from naslib.utils import nb101_api as api

* We need **nasbench_full.tfrecord** to sample architectures, if not installed yet install it under **naslib/data** from following link: https://storage.googleapis.com/nasbench/nasbench_full.tfrecord 

In [3]:
#give data format and dataset path 
nasbench = api.NASBench("/Users/safayilmaz/Desktop/DI LAB/NASLib/naslib/data/nasbench_full.tfrecord", 
                        data_format = "tfrecord")

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 82 seconds


In [4]:
from pyDOE import *
import matplotlib.pyplot as plt# Standard imports
import copy
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns 
import pickle as pkl
import time
import json

# Fixing random state for reproducibility
np.random.seed(19680801)

In [5]:
# Useful constants
INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2   # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2   # Input/output vertices are fixed
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1]   # Binary adjacency matrix

In [6]:
MAX_NODES = 7
MAX_EDGES = 9
NASBENCH_TFRECORD = './data/nasbench_full.tfrecord'

INPUT = 'input'
OUTPUT = 'output'
CONV1X1 = 'conv1x1-bn-relu'
CONV3X3 = 'conv3x3-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'

CODING = [INPUT]
CODING = CODING + [CONV1X1 + "_" + str(i) for i in range(0, (MAX_NODES - 2))]
CODING = CODING + [CONV3X3 + "_" + str(i) for i in range(0, (MAX_NODES - 2))]
CODING = CODING + [MAXPOOL3X3 + "_" + str(i) for i in range(0, (MAX_NODES - 2))]
CODING = CODING + [OUTPUT]

In [7]:
def sample_single_op(NUM_VERTICES, ALLOWED_OPS):
    x = lhs(NUM_VERTICES, samples=1)[0]
    vv = np.floor(x * len(ALLOWED_OPS))
    op  = [ALLOWED_OPS[int(k)] for k in vv]
    op[0] = INPUT
    op[-1] = OUTPUT
    return op

def sample_single_configurations_lhs(N_dimensions):    
    
    sum_edges = 0   
    while sum_edges != 9:
        v_m = lhs(N_dimensions, samples=1)
        idx = v_m > 0.5
        v_m[idx == True] = 1
        v_m[idx == False] = 0
        sum_edges = sum(v_m[0])
    
    return v_m[0]

def recover_incidence_matrix(a0, N_l=7):
    
    mat = np.zeros((N_l, N_l))
    idx_new = 0
    idx_old = 0
    
    for i in range(N_l):
        idx_new += N_l - i - 1
        
        values = a0[idx_old:idx_new]
        idx_old = idx_new
        mat[i, i+1: ] = values
        
    return mat

def sample_single_valid_spec(NUM_vert, allowed_ops):
    
    is_valid = False
    while not is_valid:
        current_op = sample_single_op(NUM_vert, allowed_ops)
        current_config = sample_single_configurations_lhs(N_dimensions = 7 * 3) 
        current_mat = recover_incidence_matrix(current_config, N_l=NUM_vert)
        current_spec = api.ModelSpec(matrix=current_mat, ops=current_op)        
        is_valid = nasbench.is_valid(current_spec)
        
    return current_mat, current_op, current_spec

def LHS_sample_N_valid_specs(N, NUM_vert, allowed_ops, nb):
    all_specs = []
    set_mats = set()
    while len(set_mats) < N:
        s = sample_single_valid_spec(NUM_vert, allowed_ops)
        m, o, sp = s
        t_m = tuple(m.reshape(NUM_vert * NUM_vert))
        hash_m = nb._hash_spec(sp)
        if hash_m not in set_mats:
             set_mats = set_mats | set([t_m])
             #all_specs.append(s)
             all_specs.append((m.tolist(), o))
    return all_specs

In [8]:
def rename_ops(ops):
    c1x1 = 0
    c3x3 = 0
    mp3x3 = 0
    new_ops = []
    for op in ops:
        if op == CONV1X1:
            new_ops = new_ops + [op + "_" + str(c1x1)]
            c1x1 = c1x1 + 1
        elif op == CONV3X3:
            new_ops = new_ops + [op + "_" + str(c3x3)]
            c3x3 = c3x3 + 1
        elif op == MAXPOOL3X3:
            new_ops = new_ops + [op + "_" + str(mp3x3)]
            mp3x3 = mp3x3 + 1
        else:
            new_ops = new_ops + [op]
    return new_ops



def encode_matrix_fixed_long(adj_matrix, ops):
    enc_matrix = np.zeros((len(CODING), len(CODING)))
    pos = [CODING.index(op) for op in ops]
    trans = dict()
    for i, ix in enumerate(pos):
        trans[i] = ix
    i, j = np.nonzero(adj_matrix)
    ix = [trans.get(n) for n in i]
    jy = [trans.get(n) for n in j]
    for p in zip(ix, jy):
        enc_matrix[p] = 1
    # Flattenning in C (row-major) ording
    encoded =  enc_matrix.flatten('C') 
    #enc_matrix[np.triu_indices(len(CODING), k=1)]
    return encoded.astype(int)


def rename_ops_fixed(ops):
    c1x1 = 0
    c3x3 = 0
    mp3x3 = 0
    new_ops = []
    for op in ops:
        if op == CONV1X1:
            new_ops = new_ops + [op + "_" + str(c1x1)]
            c1x1 = c1x1 + 1
        elif op == CONV3X3:
            new_ops = new_ops + [op + "_" + str(c3x3)]
            c3x3 = c3x3 + 1
        elif op == MAXPOOL3X3:
            new_ops = new_ops + [op + "_" + str(mp3x3)]
            mp3x3 = mp3x3 + 1
        else:
            new_ops = new_ops + [op]
    return new_ops

In [9]:
N_samples = 10


In [10]:
extra_specs = LHS_sample_N_valid_specs(N_samples, 7, ALLOWED_OPS, nasbench)

In [11]:
print(extra_specs)

[([[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0], [0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], ['input', 'maxpool3x3', 'maxpool3x3', 'conv1x1-bn-relu', 'maxpool3x3', 'maxpool3x3', 'output']), ([[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], ['input', 'maxpool3x3', 'maxpool3x3', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'output']), ([[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], ['input',